<a href="https://colab.research.google.com/github/andy8744/tensorflow-certification-cheat-sheet/blob/main/01_Image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import shutil
import os
import matplotlib.pyplot as plt
plt.style.use("dark_background")

### View random image

In [ ]:
import matplotlib.image as mpimg
import random

def view_random_image(target_dir, target_class):
  target_folder = os.path.join(target_dir, target_class)
  random_image = random.sample(os.listdir(target_folder), 1)

  img = mpimg.imread(os.path.join(target_folder, random_image[0]))
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off");

  print(f"Image shape: {img.shape}")
  return img

In [ ]:
import pathlib
import PIL

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

### Dataset From Directory
https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

In [ ]:
image_size = (224, 224)

train_dataset = keras.utils.image_dataset_from_directory(
    "./dataset", 
    image_size=image_size,
    label_mode="int",
    validation_split=0.2,
    subset="training",
)

val_dataset = keras.utils.image_dataset_from_directory(
    "./dataset",
    image_size=image_size,
    label_mode="int",
    validation_split=0.2,
    subset="validation",
)

train_dataset = train_dataset.cache().prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(tf.data.AUTOTUNE)

#### Use shutil to create directory for dataset from directory

In [ ]:
for index, data in tqdm(test_df.iterrows()):
    label = data.label
    src = os.path.join(test_src, data.filename)
    dest = os.path.join(test_dir, label, data.filename)
    shutil.copy(src, dest)

In [ ]:
for filename in tqdm(os.listdir("../input/fingers/train")):
    src = os.path.join("../input/fingers/train", filename)
    label = filename.split("_")[1].split(".")[0]
    dest = os.path.join("./train", label, filename)
    shutil.copy(src, dest)

In [ ]:
for subdir in os.listdir(SPLIT_DIR):
    print(subdir, len(os.listdir(os.path.join(SPLIT_DIR, subdir))))

#### Flow from directory

In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=train_df,
directory="../input/fingers/train",
x_col="filename",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(128,128))

### Data Augmentation Layer

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.2),
        layers.RandomHeight(0.2),
        layers.RandomWidth(0.2),
    ]
)

### Model Backbone (USE TRANSFER LEARNING!)

In [ ]:
model = tf.keras.Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.4),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model = keras.Sequential([
    layers.Rescaling(scale=1./255), # rescale between 0 and 1
    
    layers.Conv2D(32, 5, activation="relu"),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),  
    layers.Dropout(0.25),

    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),  
    layers.Dropout(0.25),

    layers.Conv2D(128, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),  
    layers.Dropout(0.25),
    
    layers.Flatten(),
    
    layers.Dense(512, activation="relu"),
    layers.BatchNormalization(), 
    layers.Dropout(0.5),
  
    layers.Dense(10, activation="softmax") 
])

### Transfer Learning

#### Efficientnet B0 (sometimes works)

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=input_shape, name="input_layer")
x = data_augmentation(inputs)
x = base_model(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax", name="output_layer")(x)

model = keras.Model(inputs, outputs)

#### Xception

In [ ]:
base_model = keras.applications.Xception(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False
inputs = keras.Input(shape=(150, 150, 3))
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(3, activation="softmax")(x)
model = keras.Model(inputs, outputs)

#train
base_model.trainable = True
#train again lower lr

#### MobileNetV2 

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image

base = MobileNetV2(weights='imagenet', include_top=False)
base.trainable = False

model = Sequential([
    layers.Rescaling(scale=1.0 / 255.0, offset=-1),
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes)
])


### Model Compiling and Fitting

In [ ]:
# For Binary
METRICS = [
tf.keras.metrics.AUC(name='roc-auc'),
tf.keras.metrics.BinaryAccuracy(name='accuracy'),
tf.keras.metrics.Precision(name='precision'),
tf.keras.metrics.Recall(name="recall")
          ]

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint

callbacks = [
	EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
	ReduceLROnPlateau(monitor='val_loss', min_lr=1e-7, patience=2, mode='min', verbose=1, factor=0.1),
	ModelCheckpoint(monitor='val_loss', filepath='./best_model.h5', save_best_only=True)
]

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.01),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    #metrics=METRICS
    metrics=["accuracy"]
)

history = model.fit(train_dataset, epochs=100, batch_size=32,
					callbacks=callbacks, validation_data=val_dataset)

### Predict on new data

In [ ]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)